# Using Tensorflow Servning to deploy a trained Keras MobileNet V2 model 

I am going to walk you through a complete a guideline of how to:

**Part 1:**
- Prepare data for training 
- Trained a deep learning model

**Part 2:**
- Serve the model with Tensorflow serving.
- Deploy to [Heroku](https://www.heroku.com/).

**In order to fully benefits from this services:**
- You should be familiar with python.
- You should have some understanding of what deep learning and neural network is.


## Prepare data for training

For the sake of simplicity, I am going to use the fashion mnist dataset because it is already optimized and labeled for a classification problem. 
It has 70,000 images, and each image is a grayscale of the size (28X28). 
which are categories into the following:

|Lable|Description|  
|-  |      -      |
| 0 | T-shirt/top | 
| 1 | Trouser     | 
| 2 | Pullover    |  
| 3 | Dress       | 
| 4 | Coat        | 
| 5 | Sandal      | 
| 6 | Shirt       |
| 7 | Sneaker     | 
| 8 | Bag         | 
| 9 | Ankle boot  | 


Let's start downloading the dataset. 
fortunately, many deep learning (DL) frameworks support fashion mnist dataset out of the box, 
including keras in our case.

In [1]:
import keras

# Load the fashion-mnist train data and test data
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

print("x_train shape:", x_train.shape, "y_train shape:", x_test.shape)


/Users/MoAir/anaconda/envs/g/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


4423680/4422102 [==============================] - 4s 1us/step
x_train shape: (60000, 28, 28) y_train shape: (10000, 28, 28)


By default `keras.datasets.fashion_mnist.load_data()` is returning training and testing dataset. 
It is essential to split the dataset into training and  testing. 
Train data: This data actually to train the neural network (NN) 
Test: To valid the NN during the traning phase, by tuning and re-adjust the hyperparameters. 
hyperparameter is a parameter whose value is set before the learning process begins.(we will see them later)

**However sometime we see train, valid and test dataset, why?**

Glad you ask!

After training the NN, we run the training NN against a our validation dataset 
to make sure that the model is generalized and is not overfitting. Overfitting basically means a model only predict the right result with the training data. However, if a model predicts the incorrectly result for the training data it's called underfitting. Here is a nice explination of [overfitting and underfitting](https://medium.com/greyatom/what-is-underfitting-and-overfitting-in-machine-learning-and-how-to-deal-with-it-6803a989c76).


If the validation result was not pleased, we retrain the NN with different hyperparameters and we repeat this until we reach to a satisfied result. Now if the tranined and validation dataset have the result or score we want, we would think we are done. Not so fast!. Since we adjust the hyperparameters, we could lead the model to overfit in both train and valid dataset. To avoid this, we split the dataset into Train, Valid, and Test. 
After getting the desired score in both train and valid data, we run the model against the test dataset. Usually the test dataset should be closed to the validation result. For example getting 90% in valid data, and 87% in testing is acceptable.

### Data normalization
TDOD: add short explanation of data normalization and link to longer article on what, why data normalization is important

In [2]:
## We then normalize the data dimensions so that they are of approximately the same scale.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

## Train a deep learning Model

I have chosn MobileNet V2 model becuase its faster to train and it is small in size. [Documentation for Individual Models](https://keras.io/applications/#documentation-for-individual-models) 

In [ ]:
from keras.applications.mobilenetv2 import MobileNetV2

model = MobileNetV2(include_top=False)
